In [ ]:
#2

In [5]:
import requests
import bs4
import os
import re
from selenium import webdriver
from selenium.webdriver.common.keys import Keys
from selenium.webdriver.common.by import By

In [6]:
#downloads a list of links to the root_folder with the prefix specified followed by a numerical index and the appropriate extension
def download_links(root_folder, links, prefix):
    if len(links) > 1:
        folder_name = os.path.join(root_folder, str(i + 1))
        os.makedirs(folder_name, exist_ok=True)
    elif len(links) == 1:
        folder_name = root_folder
        os.makedirs(folder_name, exist_ok=True)
    file_count = 1
    for link in links:
        link = link.get_attribute('src')
        res = requests.get(link)
        res.raise_for_status()

        extension_regex = re.compile(r'^.*\.(.*)$')
        extension = extension_regex.search(link)[1]
        if len(links) > 1:
            file_name = os.path.join(folder_name, prefix + str(file_count) + '.' + extension)
        else:
            file_name = os.path.join(folder_name, prefix + str(i + 1) + '.' + extension)
        with open(file_name, 'wb') as file:
            for chunk in res.iter_content(100000):
                file.write(chunk)
        print(file_name + ' created.')
        file_count += 1

In [7]:
search_term = input('Enter a search term to scrape Imgur.com: ')
print()
res = requests.get('https://imgur.com/search?q=' + search_term)
res.raise_for_status()

soup = bs4.BeautifulSoup(res.text, 'html.parser')
album_links = ['https://imgur.com' + tag.get('href') for tag in soup.select('a[class="image-list-link"]')]

browser = webdriver.Firefox()
for i, album_link in enumerate(album_links):
    print('Searching ' + album_link + '...')
    browser.get(album_link)
    
    #image
    image_links = browser.find_elements(by=By.CLASS_NAME, value='image-placeholder')
    download_links(search_term, image_links, 'img')
        
    #video
    video_links = browser.find_elements(by=By.CSS_SELECTOR, value='source[type="video/mp4"]')
    download_links(search_term, video_links, 'vid')
    
    print()

print('Done.')
browser.quit()

Enter a search term to scrape Imgur.com: minecraft

Searching https://imgur.com/gallery/cT9ov...
minecraft\img1.jpeg created.

Searching https://imgur.com/gallery/OLip1...
minecraft\2\img1.png created.
minecraft\2\img2.png created.
minecraft\2\img3.png created.
minecraft\2\img4.png created.

Searching https://imgur.com/gallery/HhKqC...
minecraft\3\img1.jpeg created.
minecraft\3\img2.png created.
minecraft\3\img3.png created.

Searching https://imgur.com/gallery/4wCqn44...
minecraft\img4.jpeg created.

Searching https://imgur.com/gallery/jYvaxQ1...
minecraft\img5.jpeg created.

Searching https://imgur.com/gallery/g5UCAeA...
minecraft\img6.jpeg created.

Searching https://imgur.com/gallery/2AC3Zet...
minecraft\vid7.mp4 created.

Searching https://imgur.com/gallery/KtHQ5...
minecraft\img8.jpeg created.

Searching https://imgur.com/gallery/e0ujj...
minecraft\9\img1.jpeg created.
minecraft\9\img2.jpeg created.

Searching https://imgur.com/gallery/kta1OVX...
minecraft\img10.png created.

Sea

NoSuchWindowException: Message: Browsing context has been discarded
Stacktrace:
WebDriverError@chrome://remote/content/shared/webdriver/Errors.jsm:188:5
NoSuchWindowError@chrome://remote/content/shared/webdriver/Errors.jsm:442:5
assert.that/<@chrome://remote/content/shared/webdriver/Assert.jsm:493:13
assert.open@chrome://remote/content/shared/webdriver/Assert.jsm:155:4
GeckoDriver.prototype.findElements@chrome://remote/content/marionette/driver.js:1531:15
despatch@chrome://remote/content/marionette/server.js:304:40
execute@chrome://remote/content/marionette/server.js:277:16
onPacket/<@chrome://remote/content/marionette/server.js:250:20
onPacket@chrome://remote/content/marionette/server.js:251:9
_onJSONObjectReady/<@chrome://remote/content/marionette/transport.js:502:20


In [ ]:
#3

In [9]:
from selenium import webdriver
from selenium.webdriver.common.keys import Keys
from selenium.webdriver.common.by import By
import itertools
import shelve
import pyinputplus as pyip

In [2]:
#initialize browser
game_browser = webdriver.Firefox()
game_browser.get('https://play2048.co/')

game = game_browser.find_element(by=By.TAG_NAME, value='body')
game_over = game_browser.find_element(by=By.CSS_SELECTOR, value='.game-message')
replay_button = game_browser.find_element(by=By.CSS_SELECTOR, value='.restart-button')
game_score = game_browser.find_element(by=By.CSS_SELECTOR, value='.score-container')

#length of key input algorithm, must be at least 4 to make sure every key occurs at least once
algorithm_length = 4

#number of trials to perform for each algorithm
num_trials = 100

scores = {}

#an algorithm is a list of tuples (plain English, keyboard code); Cartesian product of keys is taken, combinations that lack one or more key are excluded
algorithms = list(itertools.product([('up', Keys.UP), ('right', Keys.RIGHT), ('down', Keys.DOWN), ('left', Keys.LEFT)], repeat=algorithm_length))
algorithms = [algorithm for algorithm in algorithms if len(set(algorithm)) == 4]

for algorithm in algorithms:
    plain_english = [value[0] for value in algorithm]
    plain_english = tuple(plain_english)
    keyboard_codes = [value[1] for value in algorithm]
    
    print('Running algorithm ' + str(plain_english) + ' ...')
    scores[plain_english] = []
    for i in range(num_trials):
        print('\tTrial ' + str(i + 1) + ': ', end='')
        while True:
            for code in keyboard_codes:
                game.send_keys(code)
            if game_over.is_displayed():
                score = game_score.text
                print(score)
                scores[plain_english].append(int(game_score.text))
                replay_button.click()
                break
    average = sum(scores[plain_english]) / len(scores[plain_english])
    print('Average: ' + str(average))
    print()

game_browser.quit()

Running algorithm ('up', 'right', 'down', 'left') ...
	Trial 1: 2784
	Trial 2: 

WebDriverException: Message: Failed to decode response from marionette


In [7]:
#save data from Cartesian product algorithms
with shelve.open('game_algorithm_performance_data') as shelf:
    shelf['length_' + str(algorithm_length) + '_scores'] = scores

NameError: name 'scores' is not defined

In [9]:
with shelve.open('game_algorithm_performance_data') as shelf:
    scores = shelf['length_4_scores']
    
scores

{('up', 'right', 'down', 'left'): [1316,
  1368,
  676,
  2636,
  1632,
  1692,
  728,
  1232,
  4684,
  3056,
  2476,
  312,
  2924,
  1496,
  3112,
  2360,
  2872,
  3072,
  1508,
  2788,
  1624,
  1380,
  2516,
  1440,
  2876,
  2316,
  1508,
  696,
  2016,
  1740,
  2412,
  1112,
  1336,
  1016,
  1376,
  2780,
  3508,
  2032,
  3124,
  3796,
  1380,
  4448,
  2324,
  3180,
  1288,
  3076,
  1620,
  2788,
  3228,
  1128,
  1288,
  1404,
  1324,
  1360,
  1340,
  4768,
  2820,
  1368,
  2280,
  1564,
  1508,
  2428,
  3176,
  1072,
  1516,
  724,
  5200,
  1512,
  1348,
  2420,
  1524,
  1220,
  2872,
  2744,
  2480,
  1052,
  3164,
  1320,
  3492,
  1692,
  2808,
  1332,
  1412,
  2668,
  1592,
  1648,
  2540,
  5396,
  3036,
  2812,
  2000,
  2732,
  3124,
  2860,
  1324,
  720,
  4572,
  2764,
  2664,
  864],
 ('up', 'right', 'left', 'down'): [1668,
  1372,
  964,
  560,
  592,
  1136,
  812,
  688,
  1280,
  2160,
  596,
  876,
  1308,
  672,
  1344,
  768,
  1288,
  580,
  1256

In [11]:
#get user algorithm
user_algorithm = []
user_string = []
user_options = ['up', 'right', 'down', 'left', 'done']

while True:
    user_input = pyip.inputChoice(user_options, '\nEnter up, right, down, left, or done: ')
    if user_input == 'up':
        user_algorithm.append(Keys.UP)
        user_string.append(user_input)
    elif user_input == 'right':
        user_algorithm.append(Keys.RIGHT)
        user_string.append(user_input)
    elif user_input == 'down':
        user_algorithm.append(Keys.DOWN)
        user_string.append(user_input)
    elif user_input == 'left':
        user_algorithm.append(Keys.LEFT)
        user_string.append(user_input)
    elif user_input == 'done':
        if len(set(user_algorithm)) == 4:
            break
        else:
            print('\nAlgorithm must contain each direction at least once, try again.')
            user_input = []
            user_string = []
            
user_string = '-'.join(user_string)
        
    
#initialize browser
game_browser = webdriver.Firefox()
game_browser.get('https://play2048.co/')

game = game_browser.find_element(by=By.TAG_NAME, value='body')
game_over = game_browser.find_element(by=By.CSS_SELECTOR, value='.game-message')
replay_button = game_browser.find_element(by=By.CSS_SELECTOR, value='.restart-button')
game_score = game_browser.find_element(by=By.CSS_SELECTOR, value='.score-container')

while True:
    for i in range(len(user_algorithm)):
        game.send_keys(user_algorithm[i])
    if game_over.is_displayed():
        break

score = game_score.text
game_browser.quit()
print('\nYour algorithm ' + user_string + ' scored ' + str(score) + ' points.')
        


Enter up, right, down, left, or done: left

Enter up, right, down, left, or done: down

Enter up, right, down, left, or done: right

Enter up, right, down, left, or done: up

Enter up, right, down, left, or done: up

Enter up, right, down, left, or done: up

Enter up, right, down, left, or done: up

Enter up, right, down, left, or done: right

Enter up, right, down, left, or done: down

Enter up, right, down, left, or done: done

Your algorithm left-down-right-up-up-up-up-right-down scored 1336 points.


In [ ]:
#4

In [1]:
import pyinputplus as pyip
import bs4
import requests
import re

In [2]:
url = pyip.inputURL('\nEnter a URL: ')
http = 'http://'
https = 'https://'
http_regex = re.compile('^(' + http + '|' + https + ')')

if http_regex.search(url) is None:
    url = http + url

res = requests.get(url)
res.raise_for_status()

soup = bs4.BeautifulSoup(res.text, 'html.parser')
links = [tag.get('href') for tag in soup.select('a')]

good_links = []
bad_links = []

for link in links:
    if http_regex.search(link) is None:
        link = url + '/' + link
    if link in good_links or link in bad_links:
        continue
    res = requests.get(link)
    try:
        res.raise_for_status()
        good_links.append(link)
    except:
        bad_links.append(link)
        
for link in good_links:
    print(link)
    
print()

for link in bad_links:
    print('Broken link: ' + link)


Enter a URL: made2report.com
http://made2report.com/#content
https://made2report.com/
https://made2report.com/contact/
https://made2report.com/about/
http://made2report.com/#
https://made2report.com/index.php/contact/
https://made2report.com/index.php/custom-web-reports/
https://made2report.com/index.php/custom-web-apps/
https://made2report.com/new-m2m-apps/

